# Orbit Propagation

This tutorial demonstrates how to perform orbit propagation using the propagated orbit mode.

## Setup

In [ ]:
import numpy as np
import pandas as pd

import plotly.graph_objs as go

from ostk.mathematics.objects import RealInterval
from ostk.mathematics.geometry.d3.objects import Cuboid
from ostk.mathematics.geometry.d3.objects import Point

from ostk.physics.units import Mass
from ostk.physics.units import Length
from ostk.physics.units import Derived
from ostk.physics.units import Angle
from ostk.physics.units import Time
from ostk.physics.time import Scale
from ostk.physics.time import Instant
from ostk.physics.time import Duration
from ostk.physics.time import Interval
from ostk.physics.time import DateTime
from ostk.physics.coordinate import Position
from ostk.physics.coordinate import Velocity
from ostk.physics.coordinate import Frame
from ostk.physics import Environment
from ostk.physics.environment.objects.celestial_bodies import Earth
from ostk.physics.environment.objects.celestial_bodies import Sun
from ostk.physics.environment.objects.celestial_bodies import Moon

from ostk.astrodynamics import NumericalSolver
from ostk.astrodynamics.flight import SatelliteSystem
from ostk.astrodynamics.flight.system import SatelliteDynamics
from ostk.astrodynamics.trajectory import State
from ostk.astrodynamics.trajectory import Orbit
from ostk.astrodynamics.trajectory.orbit.models import Propagated

---

## Propagated Orbit Model

### Computation

Create an environment with the desired planets (this controls which perturbations can be toggled later in the propagated object, this will be fixed)

In [ ]:
earth = Earth.WGS84() # Change this to change the fidelity of the tabulated Earth gravity model
environment = Environment(Instant.J2000(), [earth])

Create an instant in time and a starting state at that instant, with respect to the GCRF (Earth-Centered-inertial) reference frame

In [ ]:
start_instant = Instant.date_time(DateTime.parse("2021-03-20T00:00:00.000"), Scale.UTC)

frame = Frame.GCRF()
position = Position.meters([-1514668.9408102570269,  -192084.12149140036718, 6831711.4584368728174], frame)
velocity = Velocity.meters_per_second([-6348.0791876050259859, 3867.5824926981121621, -1297.1761044290490705], frame)

state = State(start_instant, position, velocity)

epoch = Instant.date_time(DateTime.parse("2021-06-30T17:26:46.742"), Scale.UTC)

Create a satellite system, a satellite dynamical system, and a numerical solver

In [ ]:
mass = Mass(90.0, Mass.Unit.Kilogram)
inertia_tensor = np.ndarray(shape=(3, 3))
satellite_geometry = Cuboid(Point(0.0, 0.0, 0.0), [ [1.0, 0.0, 0.0 ], [ 0.0, 1.0, 0.0 ], [ 0.0, 0.0, 1.0 ] ], [1.0, 0.0, 0.0 ] )
surface_area = 0.8
drag_coefficient = 2.2

satellitesystem = SatelliteSystem(mass, inertia_tensor, satellite_geometry, surface_area, drag_coefficient)

satellitedynamics = SatelliteDynamics(Environment.default(),satellitesystem, state, epoch, SatelliteDynamics.GravitationalPerturbationType.No,
                    SatelliteDynamics.AtmosphericPerturbationType.No, SatelliteDynamics.ThirdBodyPerturbationType.No, SatelliteDynamics.StateVectorDimension.Position_velocity)

numericalsolver = NumericalSolver(NumericalSolver.LogType.No_log, NumericalSolver.StepperType.Runge_Kutta_Cash_Karp_54, 5.0, 1.0e-15, 1.0e-15)

Setup a Propagated model an an instant array at which propagated states are desired

In [ ]:

propagated_model = Propagated(satellitedynamics, numericalsolver)

prop_duration = Duration.seconds(11345.0) 
instant_array = [start_instant + prop_duration]

Setup the orbit (depracated for now):

In [ ]:
# orbit_default = Orbit(propagated_model_default, earth_env)
# orbit_custom = Orbit(propagated_model_custom, earth_env)

Now that everything is set up, we can calculate the state arrays from the desired time instant grid

In [ ]:
# tic
propagated_state_array = propagated_model.calculate_state_array_at_instants(instant_array)
# toc

# orbit_default.get_state_at(current_instant + propagation_duration) 

In [ ]:
def convert_state (instant, state):
    
    lla = LLA.cartesian(state.get_position().in_frame(Frame.ITRF(), state.get_instant()).get_coordinates(), Earth.equatorial_radius, Earth.flattening)
    
    return [
                repr(instant),
                float(instant.get_modified_julian_date(Scale.UTC)),
                *state.get_position().get_coordinates(),
                *state.get_velocity().get_coordinates(),
                float(lla.get_latitude().in_degrees()),
                float(lla.get_longitude().in_degrees()),
                float(lla.get_altitude().in_meters())
            ]

In [ ]:
orbit_data = [convert_state(instant, state) for [instant, state] in states]

In [ ]:
orbit_df = pd.DataFrame(orbit_data, columns=['$Time^{UTC}$', '$MJD^{UTC}$', '$x_{x}^{ECI}$', '$x_{y}^{ECI}$', '$x_{z}^{ECI}$', '$v_{x}^{ECI}$', '$v_{y}^{ECI}$', '$v_{z}^{ECI}$', '$Latitude$', '$Longitude$', '$Altitude$'])

### Graphical Orbit Output on top of Earth

Table:

In [ ]:
orbit_df.head()

2D plot, over **World Map**:

In [ ]:
figure = go.Figure(
    data = go.Scattergeo(
        lon = orbit_df['$Longitude$'],
        lat = orbit_df['$Latitude$'],
        mode = 'lines',
        line = go.scattergeo.Line(
            width = 1,
            color = 'red'
        )
    ),
    layout = go.Layout(
        title = None,
        showlegend = False,
        height=1000,
        geo = go.layout.Geo(
            showland = True,
            landcolor = 'rgb(243, 243, 243)',
            countrycolor = 'rgb(204, 204, 204)'
        )
    )
)

figure.show()

3D plot, in **Earth Fixed** frame:

In [ ]:
figure = go.Figure(
    data = [
        go.Scattergeo(
            lon = orbit_df['$Longitude$'],
            lat = orbit_df['$Latitude$'],
            mode = 'lines',
            line = go.scattergeo.Line(
                width = 2,
                color = 'rgb(255, 62, 79)'
            )
        )
    ],
    layout = go.Layout(
        title = None,
        showlegend = False,
        width = 800,
        height = 800,
        geo = go.layout.Geo(
            showland = True,
            showlakes = True,
            showcountries = False,
            showocean = True,
            countrywidth = 0.0,
            landcolor = 'rgb(100, 100, 100)',
            lakecolor = 'rgb(240, 240, 240)',
            oceancolor = 'rgb(240, 240, 240)',
            projection = dict( 
                type = 'orthographic',
                rotation = dict(
                    lon = -100,
                    lat = 40,
                    roll = 0
                )            
            ),
            lonaxis = dict( 
                showgrid = True,
                gridcolor = 'rgb(102, 102, 102)',
                gridwidth = 0.5
            ),
            lataxis = dict( 
                showgrid = True,
                gridcolor = 'rgb(102, 102, 102)',
                gridwidth = 0.5
            )
        )
    )
)

figure.show()

3D plot, in **Earth Inertial** frame:

In [ ]:
theta = np.linspace(0, 2 * np.pi, 30)
phi = np.linspace(0, np.pi, 30)

theta_grid, phi_grid = np.meshgrid(theta, phi)

r = float(Earth.equatorial_radius.in_meters())

x = r * np.cos(theta_grid) * np.sin(phi_grid)
y = r * np.sin(theta_grid) * np.sin(phi_grid)
z = r * np.cos(phi_grid)

earth = go.Surface(
    x=x,
    y=y,
    z=z,
    colorscale='Viridis',
    showscale=False
)

trace = go.Scatter3d(
    x=orbit_df['$x_{x}^{ECI}$'],
    y=orbit_df['$x_{y}^{ECI}$'],
    z=orbit_df['$x_{z}^{ECI}$'],
    mode='lines',
    marker=dict(
        size=0,
        color=orbit_df['$x_{z}^{ECI}$'],
        colorscale='Viridis',
        showscale=False
    ),
    line=dict(
        color=orbit_df['$x_{z}^{ECI}$'],
        width=1
    )
)

figure = go.Figure(
    data = [earth, trace],
    layout = go.Layout(
        title = None,
        width = 800,
        height = 1000,
        showlegend = False,
        scene = go.layout.Scene(
            xaxis = dict(
                gridcolor = 'rgb(255, 255, 255)',
                zerolinecolor = 'rgb(255, 255, 255)',
                showbackground = True,
                backgroundcolor = 'rgb(230, 230,230)'
            ),
            yaxis = dict(
                gridcolor = 'rgb(255, 255, 255)',
                zerolinecolor = 'rgb(255, 255, 255)',
                showbackground = True,
                backgroundcolor = 'rgb(230, 230,230)'
            ),
            zaxis = dict(
                gridcolor = 'rgb(255, 255, 255)',
                zerolinecolor = 'rgb(255, 255, 255)',
                showbackground = True,
                backgroundcolor = 'rgb(230, 230,230)'
            ),
            camera = dict(
                up = dict(
                    x = 0,
                    y = 0,
                    z = 1
                ),
                eye = dict(
                    x = -1.7428,
                    y = 1.0707,
                    z = 0.7100,
                )
            ),
            aspectratio = dict(x = 1, y = 1, z = 1),
            aspectmode = 'manual'
        )
    )
)

figure.show()

---